In [1]:
from model.GNN_1 import *
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from data.load_data import load_data
from torch_geometric.data import Data
import random

BATCH_SIZE = 16


list_mol_graph, properties = load_data('sars')

properties = torch.FloatTensor(properties)

print('select...')
Datas = []
for i_mol_graph, y in zip(list_mol_graph, properties):
    # print(y)
    if not y[0].isnan():
        e_index = torch.tensor([np.concatenate((i_mol_graph.start_indices, i_mol_graph.end_indices), axis=0),
                        np.concatenate((i_mol_graph.end_indices, i_mol_graph.start_indices), axis=0)]).long()
        nodes_x = torch.tensor(i_mol_graph.atom_features).float()
        e_attr = torch.tensor(np.concatenate((i_mol_graph.bond_features , i_mol_graph.bond_features), axis=0)).float()
        yi =   y[0].long() # normalize_prop(torch.tensor(properties[i]).float())
        datai = Data(x = nodes_x, edge_index=e_index, edge_attr=e_attr, y=yi)
        Datas.append(datai)

print(len(Datas))

mean = torch.mean(properties, dim=0, keepdim=True)
std = torch.std(properties, dim=0, keepdim=True)


def normalize_prop(p: torch.Tensor) -> torch.Tensor:
    return (p - mean) / std


def denormalize_prop(p: torch.Tensor) -> torch.Tensor:
    return p * std + mean


# Datas = []
# for i, i_mol_graph in enumerate(list_mol_graph):
#     # print(i_mol_graph)
#     e_index = torch.tensor([np.concatenate((i_mol_graph.start_indices, i_mol_graph.end_indices), axis=0),
#                             np.concatenate((i_mol_graph.end_indices, i_mol_graph.start_indices), axis=0)]).long()
#     nodes_x = torch.tensor(i_mol_graph.atom_features).float()
#     e_attr = torch.tensor(np.concatenate((i_mol_graph.bond_features, i_mol_graph.bond_features), axis=0)).float()
#     y = torch.tensor(properties[i]).float()  # normalize_prop(torch.tensor(properties[i]).float())
#     datai = Data(x=nodes_x, edge_index=e_index, edge_attr=e_attr, y=y)
#     Datas.append(datai)

Num_node_features = len(Datas[0].x[0])
print('Num_node_features:', Num_node_features)

print(len(Datas))
random.shuffle(Datas)

train_dataset = Datas[:int(0.7*len(Datas))]
test_dataset = Datas[int(0.7*len(Datas)):]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)



model = GCN_cls(hidden_channels=32, Num_node_features=Num_node_features, num_classes=4)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

select...


D:\Anaconda3\envs\gt\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)


9162
Num_node_features: 34
9162
Number of training graphs: 6413
Number of test graphs: 2749
GCN_cls(
  (conv1): GCNConv(34, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 32)
  (lin): Linear(in_features=32, out_features=4, bias=True)
)


In [3]:
list_mol_graph[0].bond_features[0]

array([1, 0, 0, 0, 1, 0, 1, 0, 0, 0])

In [6]:
for i in range(5):
    model.train()
    loss_all = 0
    for data in train_loader:
        optimizer.zero_grad()
        
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)
        loss_all+=loss.cpu().detach()
        # print(out[:5])
        # print(data.y[:5], '\n\n')
        
        loss.backward()
        optimizer.step()
    print("\nloss, ", loss_all)


loss,  tensor(72.5190)



loss,  tensor(72.2189)



loss,  tensor(72.4612)



loss,  tensor(71.1372)



loss,  tensor(71.2068)


In [9]:
loader = test_loader
model.eval()

correct = 0
for data in loader:  # 批遍历测试集数据集。
    out = model(data.x, data.edge_index, data.batch)  # 一次前向传播
    print(out)
    pred = out.argmax(dim=1)  # 使用概率最高的类别
    print(pred)
    correct += int((pred == data.y).sum())  # 检查真实标签
    # print(pred[:20])
    # print(data.y[:20], '\n\n')
    break
    
print('acc:',  correct / len(loader.dataset))
    # return correct / len(loader.dataset)

tensor([[  3.4265,   0.1617,  -2.0227,  -4.2056, -12.5752, -12.6479, -13.3881,
         -12.4379, -12.8911, -12.8846, -12.9445, -13.5528, -12.9231, -13.0865,
         -13.0342, -13.1879, -13.0656, -11.8983, -12.7579, -13.3960, -12.9628,
         -12.6971, -13.4097, -12.5621, -12.7188, -13.2493, -12.8496, -12.7913,
         -13.2761, -13.5727, -11.9703, -13.3187, -13.0601, -13.7608],
        [  3.8767,   0.1630,  -2.0815,  -4.1576, -13.7645, -13.8188, -14.5879,
         -13.5506, -14.1124, -14.0867, -14.1388, -14.7920, -14.1129, -14.3187,
         -14.2177, -14.4091, -14.3096, -13.0029, -13.8831, -14.6562, -14.1848,
         -13.7834, -14.6373, -13.7191, -13.9202, -14.5172, -14.0023, -14.0048,
         -14.5563, -14.8699, -13.0927, -14.5716, -14.3039, -15.0877],
        [  5.8263,   0.1337,  -2.3114,  -3.8873, -18.8378, -18.8007, -19.6974,
         -18.3127, -19.3880, -19.2612, -19.2707, -20.0614, -19.2287, -19.5870,
         -19.2323, -19.5853, -19.5476, -17.8201, -18.6788, -19.9877, -

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([[  3.7427,   0.1579,  -2.0622,  -4.1625, -13.4008, -13.4603, -14.2195,
         -13.2113, -13.7432, -13.7224, -13.7766, -14.4104, -13.7529, -13.9424,
         -13.8530, -14.0324, -13.9230, -12.6743, -13.5388, -14.2664, -13.8127,
         -13.4522, -14.2618, -13.3692, -13.5536, -14.1279, -13.6485, -13.6368,
         -14.1617, -14.4663, -12.7530, -14.1857, -13.9248, -14.6736],
        [  5.7471,   0.1054,  -2.2960,  -3.8360, -18.5715, -18.5406, -19.4183,
         -18.0644, -19.1241, -19.0018, -19.0127, -19.7680, -18.9787, -19.3107,
         -18.9579, -19.2980, -19.2440, -17.6092, -18.4265, -19.6858, -19.1578,
         -18.1979, -19.6012, -18.4565, -18.7872, -19.6221, -18.6438, -18.9581,
         -19.6854, -20.0092, -17.6821, -19.5953, -19.3494, -20.3223],
        [  6.3234,   0.0921,  -2.3634,  -3.7458, -20.0616, -20.0044, -20.9170,
         -19.4629, -20.6745, -20.5227, -20.5211, -21.3130, -20.4837, -20.8577,
         -20.

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([[  4.2585,   0.2002,  -2.1264,  -4.1997, -14.8404, -14.8653, -15.6885,
         -14.5635, -15.2380, -15.1832, -15.2203, -15.9361, -15.1792, -15.4412,
         -15.2865, -15.5231, -15.4454, -13.9909, -14.8977, -15.8104, -15.3141,
         -14.7901, -15.7573, -14.7620, -15.0049, -15.6818, -15.0551, -15.1031,
         -15.7234, -16.0595, -14.0992, -15.7143, -15.4265, -16.2979],
        [  3.3218,   0.1565,  -2.0096,  -4.2058, -12.2895, -12.3682, -13.0979,
         -12.1699, -12.5952, -12.5948, -12.6576, -13.2521, -12.6388, -12.7895,
         -12.7500, -12.8933, -12.7652, -11.6347, -12.4880, -13.0913, -12.6663,
         -12.4331, -13.1136, -12.2848, -12.4306, -12.9424, -12.5714, -12.4998,
         -12.9674, -13.2589, -11.7019, -13.0165, -12.7617, -13.4406],
        [  3.5144,   0.1533,  -2.0335,  -4.1777, -12.7907, -12.8615, -13.6023,
         -12.6394, -13.1129, -13.1038, -13.1631, -13.7722, -13.1431, -13.3092,
         -13.

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([[  4.5022,   0.1213,  -2.1463,  -4.0043, -15.3259, -15.3518, -16.1491,
         -15.0200, -15.7572, -15.6968, -15.7341, -16.3957, -15.7103, -15.9420,
         -15.7460, -15.9829, -15.8855, -14.5391, -15.3580, -16.2704, -15.8053,
         -15.2197, -16.2486, -15.2749, -15.5045, -16.1669, -15.5035, -15.6266,
         -16.2087, -16.5086, -14.5991, -16.1910, -15.9481, -16.7519],
        [  2.9519,   0.1482,  -1.9622,  -4.2288, -11.2988, -11.3953, -12.0954,
         -11.2418, -11.5741, -11.5918, -11.6626, -12.2151, -11.6498, -11.7616,
         -11.7644, -11.8741, -11.7266, -10.7172, -11.5513, -12.0386, -11.6438,
         -11.5236, -12.0892, -11.3219, -11.4302, -11.8828, -11.6092, -11.4890,
         -11.8991, -12.1749, -10.7688, -11.9710, -11.7261, -12.3330],
        [  3.4925,   0.1569,  -2.0319,  -4.1872, -12.7402, -12.8121, -13.5525,
         -12.5915, -13.0579, -13.0503, -13.1102, -13.7216, -13.0897, -13.2564,
         -13.

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([[  3.4291,   0.1548,  -2.0231,  -4.1901, -12.5688, -12.6433, -13.3790,
         -12.4314, -12.8832, -12.8780, -12.9390, -13.5419, -12.9196, -13.0790,
         -13.0272, -13.1789, -13.0552, -11.8973, -12.7523, -13.3857, -12.9534,
         -12.6882, -13.4018, -12.5578, -12.7130, -13.2394, -12.8416, -12.7858,
         -13.2669, -13.5610, -11.9668, -13.3097, -13.0543, -13.7494],
        [  3.2186,   0.1525,  -1.9966,  -4.2083, -12.0100, -12.0943, -12.8143,
         -11.9077, -12.3062, -12.3113, -12.3768, -12.9584, -12.3602, -12.4991,
         -12.4720, -12.6052, -12.4716, -11.3765, -12.2238, -12.7936, -12.3767,
         -12.1754, -12.8241, -12.0133, -12.1484, -12.6426, -12.2994, -12.2146,
         -12.6654, -12.9522, -11.4391, -12.7210, -12.4696, -13.1275],
        [  3.6417,   0.1789,  -2.0488,  -4.2206, -13.1747, -13.2325, -13.9998,
         -13.0015, -13.5155, -13.4942, -13.5467, -14.1880, -13.5180, -13.7110,
         -13.

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([[  2.8523,   0.1466,  -1.9494,  -4.2364, -11.0334, -11.1344, -11.8271,
         -10.9933, -11.3009, -11.3232, -11.3960, -11.9378, -11.3846, -11.4863,
         -11.5003, -11.6012, -11.4485, -10.4712, -11.3003, -11.7568, -11.3703,
         -11.2804, -11.8149, -11.0638, -11.1622, -11.5992, -11.3516, -11.2182,
         -11.6130, -11.8847, -10.5187, -11.6910, -11.4486, -12.0364],
        [  6.2425,   0.0976,  -2.3559,  -3.7660, -19.8627, -19.8096, -20.7193,
         -19.2769, -20.4638, -20.3165, -20.3193, -21.1082, -20.2805, -20.6514,
         -20.2358, -20.6155, -20.5781, -18.8341, -19.6485, -21.0433, -20.4914,
         -19.3828, -20.9349, -19.7234, -20.0939, -20.9971, -19.8930, -20.2847,
         -21.0678, -21.4001, -18.9107, -20.9486, -20.7034, -21.7407],
        [  4.6413,   0.1322,  -2.1668,  -4.0112, -15.7155, -15.7347, -16.5464,
         -15.3844, -16.1535, -16.0867, -16.1223, -16.8074, -16.0941, -16.3461,
         -16.

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([[  3.9068,   0.1366,  -2.0773,  -4.1010, -13.7899, -13.8430, -14.6049,
         -13.5786, -14.1578, -14.1279, -14.1790, -14.8037, -14.1580, -14.3471,
         -14.2300, -14.4187, -14.3050, -13.0726, -13.9066, -14.6605, -14.2168,
         -13.8094, -14.6624, -13.7634, -13.9498, -14.5352, -14.0196, -14.0457,
         -14.5679, -14.8624, -13.1349, -14.5841, -14.3367, -15.0743],
        [  3.2825,   0.1553,  -2.0046,  -4.2074, -12.1836, -12.2643, -12.9905,
         -12.0705, -12.4858, -12.4874, -12.5511, -13.1409, -12.5331, -12.6795,
         -12.6446, -12.7842, -12.6540, -11.5367, -12.3878, -12.9786, -12.5567,
         -12.3356, -13.0040, -12.1818, -12.3236, -12.8289, -12.4684, -12.3917,
         -12.8530, -13.1428, -11.6022, -12.9046, -12.6510, -13.3220],
        [  3.2630,   0.1573,  -2.0018,  -4.2142, -12.1360, -12.2168, -12.9434,
         -12.0264, -12.4381, -12.4398, -12.5034, -13.0927, -12.4849, -12.6306,
         -12.

In [45]:
Datas[0].edge_attr

tensor([[1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 1., 0., 0., 0.],


In [8]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[851, 34], edge_index=[2, 1774], edge_attr=[1774, 10], y=[64], batch=[851], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[785, 34], edge_index=[2, 1590], edge_attr=[1590, 10], y=[64], batch=[785], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[916, 34], edge_index=[2, 1898], edge_attr=[1898, 10], y=[64], batch=[916], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[811, 34], edge_index=[2, 1662], edge_attr=[1662, 10], y=[64], batch=[811], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[846, 34], edge_index=[2, 1730], edge_attr=[1730, 10], y=[64], batch=[846], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[815, 34], edge_index=[2, 1668], edge_attr=[1668, 10], y=[64], batch=[815], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(x=[876, 34], edge_index=[2, 1810], edge_attr=[

In [2]:
def train():
    model.train()
    
    for data in train_loader:
        optimizer.zero_grad()
        
        out = model(data.x, data.edge_index, data.batch)
        loss = F.mse_loss(out, data.y)
        
        loss.backward()
        optimizer.step()


In [32]:

# def test(loader):
loader = train_loader

model.eval()

model.eval()
    
list_pred = []
list_y = []
for data in loader:  # 批遍历测试集数据集。
    out = model(data.x, data.edge_index, data.batch)  # 一次前向传播
    # pred = float(out)  # .argmax(dim=1)                         # 使用概率最高的类别
    out = torch.flatten(out).tolist()
    list_pred.extend(out)  # 检查真实标签
    list_y.extend(data.y.tolist())

total_pred = torch.tensor(list_pred)
total_y = torch.tensor(list_y)

total_pred = denormalize_prop(total_pred)
# total_y = denormalize_prop(total_pred)

rmse = torch.sqrt(F.mse_loss(total_pred, total_y))
rmse = float(rmse)

def test(loader):
    model.eval()
    
    correct = 0
    for data in loader:                            # 批遍历测试集数据集。
        out = model(data.x, data.edge_index, data.batch) # 一次前向传播
        pred = out.argmax(dim=1)                         # 使用概率最高的类别
        correct += int((pred == data.y).sum())           # 检查真实标签
    return correct / len(loader.dataset)


D:\Anaconda3\envs\gt\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Using a target size (torch.Size([721])) that is different to the input size (torch.Size([1, 721])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


In [33]:
rmse

5.219593048095703

In [18]:
list_pred.__len__()

12

In [13]:
import time
print('begin train...')
for epoch in range(1, 100):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train rmse: {train_acc:.4f}, Test rmse: {test_acc:.4f}')
    time.sleep(0.5)
    


    

begin train...


D:\Anaconda3\envs\gt\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  
D:\Anaconda3\envs\gt\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Using a target size (torch.Size([17])) that is different to the input size (torch.Size([17, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  


Epoch: 001, Train rmse: 0.0014, Test rmse: 0.0000


Epoch: 002, Train rmse: 0.0014, Test rmse: 0.0000


KeyboardInterrupt: 